In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from modules.tfidf_custom import TfIdfCustom
from modules.tfidf import TfIdf
from modules.word_stats import WordStats
from modules.similarity import Similarity
from models.knnreg import KnnReg
from modules.data import fetch_fresh, load_pickle, save_pickle
from modules.stats import score_distribution_plot, words_count_plot, tfidf_custom_score_plot, topic_similarity_score_plot
import pandas as pd
# from scipy.stats import binned_statistic
import sys
import argparse
pickle_df = load_pickle("okt-11-1499c.p")
display(pickle_df)

,body,subreddit,submission,tfidf_score,tfidf_custom_score,length,sentences,common_word,words_count,stop_words_count,bad_words_count,bad_words,letter_count,emoticons,aspectual_class,reichenbach_tense,topic_similarity,label
0,*What if Jod was one of us?*,askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,753
1,A large chunk of my taking the lord's name in ...,askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,41889
2,Do you want a Holy War? Because that's how you...,askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,26323
3,"He would also say, “my name isn’t God” then he...",askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,6585
4,"We’d be like, “No way!” and he’d be like, “Yah...",askreddit,"What if God came down one day and said ""It's p...",0,0,0,0,0,0,0,0,,0,False,0,0,0,20301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,"As was explained elsewhere, the basic answer i...",explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1
1495,Teeth don' t just get holes.\n\nThey get littl...,explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1
1496,Bone is a living tissue with direct blood supp...,explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1
1497,Teeth aren’t bones. They are derived from ecto...,explainlikeimfive,Eli5: why is when a bone shatters it can rebui...,0,0,0,0,0,0,0,0,,0,False,0,0,0,1


In [25]:
pd.options.display.min_rows = 100
#pd.options.display.max_rows = set larger than dataset
pd.options.display.max_columns = 20
%matplotlib inline
mpl.rcParams["figure.figsize"] = 15,8

In [3]:
df = pickle_df.copy()

In [23]:

similarity = Similarity()
similarity.score(df) # "topic_similarity"

word_stats = WordStats()
df = word_stats.score(df) # "words_count", "stop_words_count", "bad_words_count", "bad_words"

tfidf_custom = TfIdfCustom()
df = tfidf_custom.score(df) # "tfidf_custom_score"

tfidf = TfIdf()
df = tfidf.score(df) # "tfidf_score"

In [50]:

inspect = df.loc[[1201]]
print(inspect[["submission"]].to_string())
print(inspect[["body"]].to_string())
display(inspect[["topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count", "bad_words", "label"]])

idxr = 1200
display(df[["body", "submission", "topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count", "bad_words", "label"]][idxr:idxr+16])

                                                      submission
1201  NY Times: Trump paid $750 in US income taxes in 2016, 2017
                                                                                                                                                                           body
1201  Thats a drop in the bucket compared to what he is robbing the government for.\n\nRemember when he promised to only charge secret service $50 to stay at trump properties?


,topic_similarity,tfidf_score,words_count,stop_words_count,bad_words_count,bad_words,label
1201,0.054277,4.02545,18.0,15.0,0.0,,132.0


,body,submission,topic_similarity,tfidf_score,words_count,stop_words_count,bad_words_count,bad_words,label
1200,>Income tax payments help finance the military...,NY Times: Trump paid $750 in US income taxes i...,0.074283,3.228242,14.0,6.0,0.0,,20.0
1201,Thats a drop in the bucket compared to what he...,NY Times: Trump paid $750 in US income taxes i...,0.054277,4.025450,18.0,15.0,0.0,,132.0
1202,Reading through the article the one thing that...,NY Times: Trump paid $750 in US income taxes i...,0.101213,6.288738,57.0,56.0,0.0,,41.0
1203,Forget EVERYTHING else he did and imagine that...,NY Times: Trump paid $750 in US income taxes i...,0.000000,4.027567,23.0,15.0,0.0,,40.0
1204,"Before anybody says ""he's smart"" and that's wh...",NY Times: Trump paid $750 in US income taxes i...,0.055460,5.147238,52.0,27.0,0.0,,54.0
1205,Top response in /conservative with almost 1000...,NY Times: Trump paid $750 in US income taxes i...,0.115998,4.767019,39.0,23.0,1.0,fuck,130.0
1206,We need an entirely new tax code. It won’t ha...,NY Times: Trump paid $750 in US income taxes i...,0.036684,4.139582,43.0,18.0,0.0,,120.0
1207,"""That's smart business sense. All billionaires...",NY Times: Trump paid $750 in US income taxes i...,0.000000,3.124706,17.0,6.0,0.0,,78.0
1208,"If Trump claims this isn't correct, he'll have...",NY Times: Trump paid $750 in US income taxes i...,0.055550,2.694468,16.0,6.0,0.0,,18.0
1209,Look how well he lives on borrowed money.,NY Times: Trump paid $750 in US income taxes i...,0.000000,2.198844,6.0,3.0,0.0,,59.0


In [6]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

feature_list = ["topic_similarity", "tfidf_score", "words_count", "stop_words_count", "bad_words_count", "bad_words"]
y = df.pop('label')
X = df[feature_list]

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=split, random_state=2)

#Create a Gaussian Classifier
gnb = GaussianNB()

# Train the model using the training sets
gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_validate)

metrics.accuracy_score(y_test, y_pred))


